In [65]:
import pandas as pd

## อ่านไฟล์ Payment

In [66]:
df = pd.read_csv("Payment_2022_2023.txt", sep="|", dtype=str) # , nrows=1000000

## สำรวจข้อมูล

In [67]:
df

,RECEIPT_NO,SERVICED_NUMBER,PAYMENT_DATE,RECEIPT_TYPE,ACCOUNT_NO,EXTERNAL_ID,NETWORK_CODE,INVOICE_DATE,INVOICE_NO,BILL_MONTH,TOTAL_AMT,DISCOUNT_AMT,PAY_TYPE,payment_type
0,10102A00120006717,028690752,04/04/2022,TaxInvABB,104531223507,NaN,TOT,21/02/2022,800306806,0265,110.21,NaN,CA,POS
1,10102A00120006717,028690752,04/04/2022,TaxInvABB,104531223507,NaN,TOT,21/03/2022,803260082,0365,107,NaN,CA,POS
2,O5504B00030012179,5578J5802,04/04/2022,TaxInv,104530287678,NaN,TOT,11/02/2022,799245256,0265,321,NaN,CA,POS
3,O5504B00030012179,5578J5802,04/04/2022,TaxInv,104530287678,NaN,TOT,11/03/2022,802411261,0365,321,NaN,CA,POS
4,60135B00140021918,024293606,04/04/2022,TaxInv,104530774924,NaN,TOT,21/01/2022,797290769,0165,212.93,NaN,CA,POS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29092244,0D014E12650600432,027465008,22/12/2022,TaxInv,104531506407,NaN,TOT,21/11/2022,0000826654535,1165,107,NaN,DD,Batch
29092245,0D014E12650600305,027465007,22/12/2022,TaxInv,104531497596,NaN,TOT,21/11/2022,0000826654229,1165,107,NaN,DD,Batch
29092246,0F806E13650144886,023795085,22/12/2022,TaxInv,005501066293,NaN,TOT,21/11/2022,0000826544917,1165,107,NaN,DC,Batch
29092247,0F806E13650144231,073323355,22/12/2022,TaxInv,000131631829,NaN,TOT,21/11/2022,0000826944290,1165,417.3,NaN,DC,Batch


## ลบบรรทัดข้อมูลที่ไม่มีข้อมูลวันที่ Invoice ออก

In [68]:
df = df.dropna(subset=['INVOICE_DATE'])

## แปลงรูปแบบข้อความให้เป็นวันที่ เพื่อใช้คำนวณระยะเวลา

In [69]:
# Specify the format string (assuming dd/mm/yyyy)
format = '%d/%m/%Y'

df['INVOICE_DATE'] = df['INVOICE_DATE'].str[:10]
df['PAYMENT_DATE'] = df['PAYMENT_DATE'].str[:10]
# Convert 'INVOICE_DATE' to datetime format
df['INVOICE_DATE'] = pd.to_datetime(df['INVOICE_DATE'], format=format)
df['PAYMENT_DATE'] = pd.to_datetime(df['PAYMENT_DATE'], format=format)

/var/folders/k6/lgrdm_ms2dq6f2946xjl4c800000gn/T/ipykernel_32640/675423043.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INVOICE_DATE'] = df['INVOICE_DATE'].str[:10]
/var/folders/k6/lgrdm_ms2dq6f2946xjl4c800000gn/T/ipykernel_32640/675423043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PAYMENT_DATE'] = df['PAYMENT_DATE'].str[:10]
/var/folders/k6/lgrdm_ms2dq6f2946xjl4c800000gn/T/ipykernel_32640/675423043.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28850427 entries, 0 to 29092247
Data columns (total 14 columns):
 #   Column           Dtype         
---  ------           -----         
 0   RECEIPT_NO       object        
 1   SERVICED_NUMBER  object        
 2   PAYMENT_DATE     datetime64[ns]
 3   RECEIPT_TYPE     object        
 4   ACCOUNT_NO       object        
 5   EXTERNAL_ID      object        
 6   NETWORK_CODE     object        
 7   INVOICE_DATE     datetime64[ns]
 8   INVOICE_NO       object        
 9   BILL_MONTH       object        
 10  TOTAL_AMT        object        
 11  DISCOUNT_AMT     object        
 12  PAY_TYPE         object        
 13  payment_type     object        
dtypes: datetime64[ns](2), object(12)
memory usage: 3.2+ GB


## หายอดรับชำระโดยไม่รวม VAT

In [71]:
# Group by ACCOUNT_NO and aggregate AMOUNT_DUE into a list
df["TOTAL_AMT"] = df["TOTAL_AMT"].astype(float)
df["AMOUNT"] = df["TOTAL_AMT"] / 1.07
grouped_df = df.groupby('ACCOUNT_NO')['AMOUNT'].apply(list).reset_index()
grouped_df
grouped_df.to_csv("ACCOUNT_NO_PAYMENT.csv", float_format="%.2f")

/var/folders/k6/lgrdm_ms2dq6f2946xjl4c800000gn/T/ipykernel_32640/3737390799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TOTAL_AMT"] = df["TOTAL_AMT"].astype(float)
/var/folders/k6/lgrdm_ms2dq6f2946xjl4c800000gn/T/ipykernel_32640/3737390799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["AMOUNT"] = df["TOTAL_AMT"] / 1.07


In [72]:
df.columns

Index(['RECEIPT_NO', 'SERVICED_NUMBER', 'PAYMENT_DATE', 'RECEIPT_TYPE',
       'ACCOUNT_NO', 'EXTERNAL_ID', 'NETWORK_CODE', 'INVOICE_DATE',
       'INVOICE_NO', 'BILL_MONTH', 'TOTAL_AMT', 'DISCOUNT_AMT', 'PAY_TYPE',
       'payment_type', 'AMOUNT'],
      dtype='object')

In [73]:
# df[df['PAYMENT_DATE'].dt.year == 2023]

### ตรวจสอบข้อมูล payment_type column

In [74]:
df[['payment_type']].value_counts().sort_values(ascending=False)

payment_type
Batch           16699314
POS             11017568
eService         1133545
Name: count, dtype: int64

In [75]:
df[['PAY_TYPE']].value_counts().sort_values(ascending=False)

PAY_TYPE
CA          19042557
DD           4352325
CQ           2704485
DC           1755590
CC            868718
PY             83257
CH             43495
Name: count, dtype: int64

In [76]:
'''
# อย่าทดสอบกับข้อมูลเยอะ

counts = df[['ACCOUNT_NO', 'INVOICE_NO']].value_counts().sort_values(ascending=False)
# Sort the Series by 'ACCOUNT_NO' level (assuming hierarchical index)
sorted_counts = counts.sort_index(level='ACCOUNT_NO')
sorted_counts
'''

"\n# อย่าทดสอบกับข้อมูลเยอะ\n\ncounts = df[['ACCOUNT_NO', 'INVOICE_NO']].value_counts().sort_values(ascending=False)\n# Sort the Series by 'ACCOUNT_NO' level (assuming hierarchical index)\nsorted_counts = counts.sort_index(level='ACCOUNT_NO')\nsorted_counts\n"

## สรุปข้อมูล หาค่าเฉลี่ยของ payment ต่อเดือน ตัด columns ที่ไม่จำเป็นออก

In [77]:
summary_df = df.groupby(['ACCOUNT_NO', 'SERVICED_NUMBER', 'PAYMENT_DATE', 
       'INVOICE_DATE',
       'PAY_TYPE',
       'payment_type'])['AMOUNT'].mean().reset_index()
summary_df

,ACCOUNT_NO,SERVICED_NUMBER,PAYMENT_DATE,INVOICE_DATE,PAY_TYPE,payment_type,AMOUNT
0,000100000098,021781046,2022-01-12,2021-12-11,DC,Batch,100.0
1,000100000098,021781046,2022-02-14,2022-01-10,DC,Batch,100.0
2,000100000098,021781046,2022-03-14,2022-02-10,DC,Batch,100.0
3,000100000098,021781046,2022-04-12,2022-03-11,DC,Batch,100.0
4,000100000098,021781046,2022-05-12,2022-04-11,DC,Batch,100.0
...,...,...,...,...,...,...,...
17021886,104533246834,4441J1226,2022-12-29,2022-08-11,CA,POS,290.0
17021887,104533246834,4441J1226,2022-12-29,2022-09-11,CA,POS,290.0
17021888,104533246834,4441J1226,2022-12-29,2022-10-11,CA,POS,290.0
17021889,104533246834,4441J1226,2022-12-29,2022-11-11,CA,POS,290.0


## หา ระยะเวลาชำระ วันที่ชำระ - วันที่ออกใบแจ้งหนี้

In [78]:
# Calculate day difference (assuming both columns are datetime format)
summary_df['DAYS_DIFF'] = (summary_df['PAYMENT_DATE'] - summary_df['INVOICE_DATE']).dt.days

In [79]:
summary_df

,ACCOUNT_NO,SERVICED_NUMBER,PAYMENT_DATE,INVOICE_DATE,PAY_TYPE,payment_type,AMOUNT,DAYS_DIFF
0,000100000098,021781046,2022-01-12,2021-12-11,DC,Batch,100.0,32
1,000100000098,021781046,2022-02-14,2022-01-10,DC,Batch,100.0,35
2,000100000098,021781046,2022-03-14,2022-02-10,DC,Batch,100.0,32
3,000100000098,021781046,2022-04-12,2022-03-11,DC,Batch,100.0,32
4,000100000098,021781046,2022-05-12,2022-04-11,DC,Batch,100.0,31
...,...,...,...,...,...,...,...,...
17021886,104533246834,4441J1226,2022-12-29,2022-08-11,CA,POS,290.0,140
17021887,104533246834,4441J1226,2022-12-29,2022-09-11,CA,POS,290.0,109
17021888,104533246834,4441J1226,2022-12-29,2022-10-11,CA,POS,290.0,79
17021889,104533246834,4441J1226,2022-12-29,2022-11-11,CA,POS,290.0,48


## เขียนข้อมูลลงไฟล์

In [80]:
summary_df.to_csv("SUMMARY_PAYMENT.csv", index=False, float_format="%.2f")

In [81]:
summary_2023_df = summary_df[summary_df['INVOICE_DATE'].dt.year == 2023]

In [82]:
summary_2023_df

,ACCOUNT_NO,SERVICED_NUMBER,PAYMENT_DATE,INVOICE_DATE,PAY_TYPE,payment_type,AMOUNT,DAYS_DIFF
7602138,011022344300,0891126123,2022-12-27,2023-01-30,CA,POS,186.915888,-34


In [83]:
summary_2023_df.to_csv("SUMMARY_PAYMENT_2023.csv", index=False, float_format="%.2f")